# [1] 確定有一評論資料夾
資料夾內為若干TXT檔，各檔的檔名是一景點的景點編號(EX: 15.txt)。

內容則各存放「單一景點」的評論

各檔內容為：「第一行」空白；「第二行」是景點的中文名稱；

           「第三行之後」每一行都是一筆評論。編碼UTF-8

# [2] 先確定有關鍵字的文字檔
打開－「Method_V2_NoEng_Write_4PKResult.ipynb」

使用－「FourWayPK()」輸出的結果(記得要寫標頭-->最後一個參數要是'T')

得到－「NE_SciTfidf」開頭的TXT檔

# [3] 把TXT檔內容讀成Python的字典
把結果寫成Python字典(KEY:景點　VALUE:關鍵字)

使用結果會產生兩字典：

(1)DictA：KEY:景點"編號"　VALUE:關鍵字

(2)DictB：KEY:景點"名稱"　VALUE:關鍵字


使用範例：

    TransWordsToDict(KeyWordsFile,Folder,PrintReport,PrintDict,ReturnDict)
   
    KeyWordsFile : [字串]。[檔名]。存放關鍵字結果的檔案( 要有標頭 )
    ----->EX: 'NE_SciTfidf-30-20160118.txt'
    
    Folder : [字串]。[資料夾名]。存放評論的資料夾( 格式要照[1]的規定 )
    ----->EX: 'CommentSample'
    
    PrintReport : [字串]。['Y'or'N']。決定是否印出報告，以確認讀取過程中有無遺漏
                  'Y'：印出報告
                  'N'：不印
    
    PrintDict   : [字串]。['PrintA'or'PrintB'or'N']。決定是否印出字典來看看
                  'PrintA'：印出DictA
                  'PrintB'：印出DictB
                  'N'：不印

    ReturnDict  : [字串]。['ReturnA'or'ReturnB'or'N']。決定是否回傳字典
                  'ReturnA'：回傳DictA
                  'ReturnB'：回傳DictB
                  'N'：不回傳
                  注意！此參數是為了可以連接其他方法使用，在使用時，
                  第 3 . 4 個參數都要是 'N' 才能使用！

In [1]:
##### [3] #####

def TransWordsToDict(KeyWordsFile,Folder,PrintReport,PrintDict,ReturnDict):
    import re
    from pprint import pprint
    total = 0#　計算總共有多少行資料
    title = 0#　計算有多少的景點
    length = 0#　計算有多少個關鍵字

    ReAttr = '('+Folder+')(.)(.*)'#　計算表頭，正規化用
    for i in open(KeyWordsFile,'r'):#　打開結果的檔案 
        if re.match(ReAttr,i.strip()):#　計算表頭(來源的資料夾)
            title += 1#　表頭(景點)＋１
        else:#　不是表頭。就是關鍵字
            length += 1#　關鍵字＋１
        total += 1#　全部＋１

    if PrintReport == 'Y':
        print "景點總數:",title," 關鍵字總數:",length," 原始檔長度:",total#　把數字都印出來
        if (title+length) == total:#　如果 (景點+關鍵字) = (總行數)
            print '1. 檔案讀取成功! 並無遺漏!\n'#　代表沒有遺漏！成功讀取
        else:
            print '1. 錯誤! 請檢查!\n'


    domain = Folder+'\\'#  景點評論所在的資料夾(同時也是分割內容的標準)

    with open(KeyWordsFile,'r') as fid:#　再讀一次結果的檔案
        content = fid.read()#　把內容讀出來，存給content 
        DictA = {}#　創建一字典
        c = 0#　另一變數c 初值為0

        for i in content.split(domain):#　把內容依據CommentSample(存放景點評論的資料夾名稱)切割
            if c > 0:#　因為split的結果，第0個(也就是for迴圈第一次進來)的值會是空的，所以至少要第二次進來的我才要
                d = 0#　另一變數d 初值為0
                Now = ''

                for o in i.split():#　用空白把每個資料切開
                    if d == 0:#　split的結果，第0個會是檔名(EX: 15.txt)，並不是關鍵字，要特別處理
                        Now = o.strip()#　如果是表頭的話，就把文字指定給外面的變數Now
                        DictA[Now] = []#　把表頭新增進字典裡，並指定內容為一陣列
                    else:#　關鍵字都會進來這裡
                        DictA[Now].append(o.strip())#　把關鍵字加進字典的Value陣列裡
                    d+=1#　做完之後，把d加一。目的僅是因為首次進來(d=0)時，要為字典新增KEY
                        #  以便之後的關鍵字有家可歸，之後繼續加上去並無實值意義
            c+=1#　做完之後，把c加一。目的僅是為了避免第一次進來(c=0)的情況，之後繼續加上去並無實值意義


    ######　　做完以上動作，會得到一字典DictA，KEY:(各景點編號.txt) Value:(各景點的關鍵字)


    DictB = {}#  創建一字典(目的是要把DictA的KEY)轉換成中文
    cnt = 0#  計算關鍵字數量用

    for a in DictA:#  把字典的KEY(EX: 15.txt)讀出來
        count = 0#  另一變數初值為 0
        for u in open(domain+a,'r'):#  (資料夾+字典的KEY) = 檔案的相對位置；把檔讀出來 
            count+=1#  每次進來都先+1，表是讀到第 N 行
            if count == 2:#  當count = 2(讀到第二行, 也就是景點名稱)時，(假設讀到15.txt, 第二行是'陽明山')
                DictB[u.strip()] = DictA[a]#  就把(15.txt)的Value指定給新的字典的'陽明山'這個KEY，達到檔名轉中文的目的
                break#  得到中文景點名稱之後，就沒有必要讀下去，直接中斷此次迴圈
        for b in DictA[a]:#  把每個KEY的關鍵字讀出來
            cnt += 1#  計算關鍵字數量

        #####印出原始字典(DictA)#####
        if PrintDict == 'PrintA':
            print a,'\n',str(DictA[a]).decode('string_escape')
        ###########################

    if PrintReport == 'Y':
        print 'DictA內含詞語總量: ',cnt
        if cnt == length:
            print '2. DictA寫入成功! 並無遺漏!\n'
        else:
            print '2. 錯誤! DictA寫入過程出現遺漏!\n'


    cnt2 = 0#  計算關鍵字數量用
    #####印出最終字典#####
    for g in DictB:
        if PrintDict == 'PrintB':
            print g,'\n',str(DictB[g]).decode('string_escape') 
    #####################
        for r in DictB[g]:
            cnt2 += 1

    if PrintReport == 'Y':
        print '\nDictB內含詞語總量: ',cnt2
        if cnt2 == cnt:
            print '3. DictB寫入(轉換)成功! 並無遺漏!'
        else:
            print '3. 錯誤! DictB寫入(轉換)過程出現遺漏!'

    if ReturnDict == 'ReturnA':
        return DictA
    elif ReturnDict == 'ReturnB':
        return DictB

# ---[測試A] 檢視TransWordsToDict()的結果

In [8]:
#---[測試A]
##########

#設定參數

KeyWordsFile = 'NE_SciTfidf-30-20160118.txt'
Folder = 'CommentSample'
PrintReport = 'Y'
PrintDict = 'N'
ReturnDict = 'N'

#呼叫方法
TransWordsToDict(KeyWordsFile,Folder,PrintReport,PrintDict,ReturnDict)


景點總數: 178  關鍵字總數: 3848  原始檔長度: 4026
1. 檔案讀取成功! 並無遺漏!

DictA內含詞語總量:  3848
2. DictA寫入成功! 並無遺漏!


DictB內含詞語總量:  3848
3. DictB寫入(轉換)成功! 並無遺漏!


# [4] 選擇使用哪個W2V模型
使用方法：

    W2V_ClassificationMethod2(Concept,Corpus,SimLevel,SimPercent,Model)
    
    Model : [字串]。
            'Wiki'   :中文WIKI
            'SogouCS':搜狐新闻数据(08年)
            'SogouCA':全网新闻数据(08年)
    
    結果會Return一個已將關鍵字分好類的字典：KEY:概念 Value:概念詞

In [2]:
##### [4] #####

def W2V_ClassificationMethod2(Concept,Corpus,SimLevel,SimPercent,Model):
    
    import gensim
    #1__WIKI
    if Model == 'Wiki':
        model = gensim.models.Word2Vec.load("W2V_Wiki_Trastd/wiki.zh.sim.seg.trastd.model")
    #2__搜狐新闻数据
    elif Model == 'SogouCS':
        model = gensim.models.Word2Vec.load('W2V_SogouCS/SogouCS_Combine.model')
    #3__全网新闻数据
    elif Model == 'SogouCA':
        model = gensim.models.Word2Vec.load('W2V_SogouCA/SogouCA_Combine.model')
    
    
    import operator
    cnt = []#　　計算餵進來的總詞量
    Expt = []#　　存W2V無法辨識的詞語
    Remain = []#　　存沒有通過門檻的詞語
    ClassingResult = {}

    for z in Concept:#　　把概念標頭寫進字典
        ClassingResult[z] = {}#　　每個字典的內容都是一個字典

    for q in open(Corpus,'r'):#　　把萃取關鍵字的結果讀出來        
        q = q.strip().decode('utf-8')#　　去空白,解碼
        cnt.append(q)#　　把讀取內容寫進一陣列(計算樣本的資料量用)
        try:
            TmpDic = {}#　　暫時性的字典(比對詞語與各概念的相似度)
            SameConcept = ''#　　用來存放概念,當q直接是某概念時使用
            TmpWord = 'None'#　　用來比對詞語是不是與概念完全吻合,若是,None會被修改掉;反之則維持None(初值)

            #print '@ ->「',q,'」的概念分類開始: '

            for a in Concept:#　　讀取各概念標頭
                TmpList = []#　　暫時性的陣列(存放'詞語'vs'概念內各詞語'的相似度)     
                for b in Concept[a]:#　　讀取個概念內的詞語

                    #####完全ㄧ樣#####
                    if q.encode('utf-8') == b:#　　如果等於
                        #print '---目標詞「',q,'」就在【',a,'】的概念裡'
                        TmpWord = b#　　把詞語指定給TmpWord
                        break#　　跳出迴圈(已經知道跟某詞語完全一樣,就不用再比其他的詞語了)
                    #####

                    if model.similarity(q,b.decode('utf-8')) > SimLevel:#　　如果相似度達門檻
                        TmpList.append(model.similarity(q,b.decode('utf-8')))#　　就把相似度寫進TmpList裡面

                #####完全一樣#####        
                if TmpWord != 'None':#　　不是None(初值)->被修改過->要完全相等才會出現此情況
                    SameConcept = a#　　把現在比對到的概念( EX:輕鬆 )記起來
                    break#　　跳出迴圈(已經知道是屬於哪個概念,就不用再比對其他概念了)
                #####

                #####印出q與一概念比較的詳細數據
                #print '  -目標詞「',q,'」與【',a,'】的相似陣列( 顯示內容為已高於相似度門檻 :',SimLevel,')'
                #print TmpList
                #print '  -其占【',a,'】概念內容的\n   比重為 :',len(TmpList),'/',len(Dic[a])
                #print '   比率為 :',
                #print float(len(TmpList))/float(len(Concept[a]))
                #print '*'
                #####

                if float(len(TmpList))/float(len(Concept[a])) > SimPercent:#　　如果達到相似成數的門檻
                    #print '  -「',q,'」已過相似_百分比_門檻 :',SimPercent
                    TmpAvg = 0.0#　　存放平均相似度
                    for c in TmpList:#　　把每個相似度讀出來
                        TmpAvg += c#　　加起來
                    TmpDic[a] = float(TmpAvg)/float(len(TmpList))#　　把'總數'除以'陣列長度',得到平均相似度,
                                                                    #再寫進字典裡,當q與多個概念都有相似度食
                                                                    #可以比較與哪個概念的相似度比較高
                    #print '   與【',a,'】的相似度為: ',TmpDic[a],' -> 進入候選名單'
                    #print '# # # # # # #\n'

            #####完全一樣##### 
            if TmpWord != 'None': 
                ClassingResult[SameConcept][(q.encode('utf-8'))] = 1#　　把q寫進總字典的概念裡(Key:q Value:1)
                continue
            #####

            #####檢視暫存字典#####
            #print '\n___檢視',q,'的暫存字典(TmpDic)___'
            #for k in TmpDic:#　　遍歷暫存字典,看q與個概念的相似度
            #    print k,TmpDic[k]
            #print '== == == == == == == == == == =='
            #####

            if len(TmpDic) > 0:#　　如果暫存字典有任何一概念(表示q'至少'與一個概念相似)
                #print '> >小結 :',q,'成功歸類！',
                for d in TmpDic:#　　遍歷TmpDic
                    if d == sorted(TmpDic.iteritems(),key=operator.itemgetter(1),reverse=True)[0][0]:
                        #如果d = 排序後的第一名 (表示d就是與q最像的概念)
                        if TmpDic[d] > SimLevel:#　　如果"q與概念的相似度"大於相似度門檻
                            #print '類別為 :',d,'\n'
                            ClassingResult[d][q.encode('utf-8')] = TmpDic[d]#　　把q寫進總字典的概念裡(Key:q Value:相似度)
                        else:#　　不太可能發生的情況
                            Remain.append('WHAT!?'+q.encode('utf-8'))                    
            else:#　　q與各概念相似度都沒有達標,無法歸類
                Remain.append(q.encode('utf-8'))

        except KeyError:#　　抓到KeyError( W2V不認識q )的例外處理
            #print '「',q,'」EXCEPT! (KeyError)\n'
            Expt.append(q.encode('utf-8'))#　　把q加進例外的陣列理
            
#####　　把分類結果字典回傳　　#####
    return ClassingResult

# [5] 讀取概念

In [3]:
##### [5] #####

Dic = {
'藝文':['藝術','美術','戲劇','音樂'],\
    
'信仰':['信仰','祭祀','祭拜','膜拜','信眾','信奉'],\
    
'美食':['美食','爽口','美味','好吃'],\
'新奇':['新奇','新穎','獵奇','獨特','驚喜'],\

'輕鬆':['輕鬆','休閒','遊憩','悠閒','休憩','消遣','愜意','放鬆'],\
'遊玩':['遊玩','好玩','遊樂'],\
'快樂':['快樂','愉快','開心','歡樂','幸福','美好'],\
'有趣':['有趣','趣味'],\

'脫俗':['脫俗','幽雅','清幽','優雅','氣質','心曠神怡'],\
'樂活':['環保','綠化','清新'],\

'安靜':['安靜','寧靜'],\
'乾淨':['乾淨','整潔','潔淨'],\

'懷舊':['懷舊','復古','情調'],\
'古老':['古老','悠久'],\

'髒亂':['骯髒','很髒','髒亂','低劣','灰塵','破舊'],\

'購物':['商圈','百貨','商場'],\
'特產':['特產','紀念品'],\

'運動':['運動','健康'],\

'漂亮':['漂亮','可愛','美麗','優美'],\
'文青':['清新','優雅','氣質'],\

'壯觀':['雄偉','壯觀','壯麗']
}

# ---[測試B] 檢視W2V_ClassificationMethod2()結果

In [9]:
#---[測試B]
##########

#把回傳結果(字詞歸類完的字典)用一個變數接著
ClassingResult = W2V_ClassificationMethod2(Dic,'NE_SciTfidf-30-20160118.txt',0.35,0.49,'Wiki')

In [10]:
#---[測試B]
##########

#可印出來看看
for ee in ClassingResult:
    print ee,str(ClassingResult[ee]).decode('string_escape')

遊玩 {'遊玩': 1, '好玩': 1, '遊樂': 1}
新奇 {'想像力': 0.43064241505868656, '豐富': 0.4509356715801665, '驚喜': 1, '表現力': 0.42818609634164245, '獨特': 1, '現代感': 0.49488485715569563}
懷舊 {'復古': 1, '情感世界': 0.4535131719488257, '悠揚': 0.4416992669416935, '別具一格': 0.4032206920639193, '懷舊': 1}
乾淨 {'舒適': 0.39928212581537637, '整潔': 1, '乾淨': 1, '水質': 0.3720850214532764}
藝文 {'戲劇': 1, '藝術': 1, '音樂': 1, '戲曲': 0.49011586151432945, '偶戲': 0.3675229526902668, '攝影': 0.5631884510979743, '雕塑': 0.47193195897590756, '花藝': 0.35236732581901803, '創意': 0.44984862638870626, '美術': 1}
信仰 {'虔誠': 0.5009075344501047, '湯守': 0.35690754287699994, '信仰': 1, '善男信女': 0.37082087632450805, '城隍': 0.47197734037115086, '宗教': 0.527963450370827, '神靈': 0.49279764271267884, '寺廟': 0.40795445270931524, '祖師': 0.43999985288167637, '金紙': 0.4065994632160217, '廟宇': 0.5482696953339619, '靈驗': 0.5032411999770633, '媽祖廟': 0.39193661420182846, '祈福': 0.49170126301354206, '神明': 0.5645932082687286, '跪拜': 0.388789287460657, '香客': 0.5321886414380881, '祭儀': 0.453899270290

# [6] 把W2V分類結果 與 各景點關鍵字 的結果接住
使用方法：

    1.先把前兩個方法的結果接住
    
    Concept   =Dic
    Corpus    ='NE_SciTfidf-30-20160118.txt'
    SimLevel  =0.35
    SimPercent=0.49
    Model     ='Wiki'
    
    ClassingResult =
    W2V_ClassificationMethod2(Concept,Corpus,SimLevel,SimPercent,Model):
    
    KeyWordsFile = 'NE_SciTfidf-30-20160118.txt'
    Folder = 'CommentSample'
    PrintReport = 'N'
    PrintDict = 'N'
    ReturnDict = 'ReturnB'
    
    DictFinal =
    TransWordsToDict(KeyWordsFile,Folder,PrintReport,PrintDict,ReturnDict)

In [4]:
##### [6] #####

Concept   =Dic
Corpus    ='NE_SciTfidf-30-20160118.txt'
SimLevel  =0.35
SimPercent=0.49
Model     ='Wiki'

ClassingResult=W2V_ClassificationMethod2(Concept,Corpus,SimLevel,SimPercent,Model)

KeyWordsFile = 'NE_SciTfidf-30-20160118.txt'
Folder = 'CommentSample'
PrintReport = 'N'
PrintDict = 'N'
ReturnDict = 'ReturnB'

DictFinal=TransWordsToDict(KeyWordsFile,Folder,PrintReport,PrintDict,ReturnDict)

# [7] 轉換成字典, 
#      把各景點的關鍵字分為'概念'跟'文字雲'擺好

In [5]:
##### [7] #####

DictResult={}#　把景點關鍵字的結果轉成'概念'.'文字雲'
for a in DictFinal:#　把{景點:[關鍵字]}的字典讀出來
    DictResult[a] = {'概　念':{},'文字雲':[]}#　指定KEY:景點 Value:概念{} 文字雲
    
    for b in DictFinal[a]:#　把每個景點的關鍵字讀出來
        z = 'Homeless'#　先假設每個關鍵字都沒有家
        
        for c in ClassingResult:#　讀取每個概念,內容是{概念:{詞語:比重}}讀出來
            if b in ClassingResult[c]:#　如果b(現在讀到的這個關鍵字)在某概念裡面
                z = 'Home'#　有家了
                
                for e in ClassingResult[c]:#　把概念的詞都讀一讀
                    if b == e:#　如果'概念內的詞'='現在的關鍵字'-->抓到了!
                        if not c in DictResult[a]['概　念']:#　如果景點還沒有此概念
                            DictResult[a]['概　念'][c]={}#　為景點的此概念新增一個字典
                            DictResult[a]['概　念'][c][b] = ClassingResult[c][e]# 並且把字詞寫進此景點的概念裡,KEY:字詞 Value:字詞與概念的相似度    
                        else:#　如果景點已經有此概念了
                            DictResult[a]['概　念'][c][b] = ClassingResult[c][e]#　把字詞寫進此景點的概念裡,KEY:字詞 Value:字詞與概念的相似度
                        break#　因為以經分好家了,寫完之後就跳出迴圈
                break#　因為已分好家,所以再次跳出迴圈
                
        if z == 'Homeless':#　如果字詞並沒有屬於任何概念,依然沒有家
            DictResult[a]['文字雲'].append(b)#　就把字詞寫進該景點的'文字雲'裡面

# ---[測試C] 驗證結果 有無遺漏
計算分類完的詞數量(DictResult裡的詞量)，並印出。若與「測試A」的結果相吻合，代表沒有出錯！

In [11]:
#---[測試C]
##########

#驗證結果
Class = 0
Cloud = 0
for g in DictResult:
    for h in DictResult[g]:
        if h == '文字雲':
            Cloud += len(DictResult[g][h])
        if h == '概　念':
            for k in DictResult[g][h]:
                Class += len(DictResult[g][h][k])

print Class + Cloud

3848


# [8] 把結果的字典印出來

In [6]:
##### [8] #####

#印出完整結果
for g in DictResult:
    print '【',g,'】'
    for h in DictResult[g]:
        print '｢',h,'｣'
        
        if h == '概　念':
            for k in DictResult[g][h]:
                print k,len(DictResult[g][h][k]),str(DictResult[g][h][k]).decode('string_escape')
        elif h == '文字雲':
            print str(DictResult[g][h]).decode('string_escape')
        
    print ''

【 新光摩天大樓 】
｢ 概　念 ｣
｢ 文字雲 ｣
['區位', '摩天大樓', '眺望臺', '特別']

【 松山慈祐宮 】
｢ 概　念 ｣
古老 1 {'繁盛': 0.35913752310115266}
購物 1 {'夜市': 0.4792209629882955}
信仰 5 {'參拜': 0.48706491755018294, '祭祀': 1, '信眾': 1, '媽祖廟': 0.39193661420182846, '廟宇': 0.5482696953339619}
運動 2 {'弱勢團體': 0.410399846769354, '關懷': 0.3974606837415023}
壯觀 1 {'精美': 0.43867902051357}
新奇 1 {'驚喜': 1}
美食 1 {'美食': 1}
有趣 1 {'厲害': 0.4151843376279619}
｢ 文字雲 ｣
['急難救助', '仁懷', '冬令救濟', '人溺己溺', '橫下心來', '獎助學金', '績優', '雕刻', '第三級', '自建', '慈悲', '廟口', '值得一試', '常年', '古蹟', '專修', '特維']

【 台北藝術大學 】
｢ 概　念 ｣
藝文 1 {'藝術': 1}
｢ 文字雲 ｣
['學位', '區及', '平原']

【 捷運北門站 】
｢ 概　念 ｣
懷舊 1 {'懷舊': 1}
｢ 文字雲 ｣
['北門', '驛站', '壓迫', '高架橋']

【 蔡瑞月舞蹈研究社 】
｢ 概　念 ｣
有趣 1 {'有趣': 1}
脫俗 1 {'幽靜': 0.4888200844111459}
｢ 文字雲 ｣
['乍眼', '研究社', '蔡瑞月', '鑄字', '日星', '夾著', '拐個', '木結構']

【 永康街青田街 】
｢ 概　念 ｣
漂亮 1 {'喜歡': 0.5172324507144339}
古老 1 {'著名': 0.37768929398901124}
樂活 1 {'清新': 1}
脫俗 1 {'流連忘返': 0.40327578900010613}
安靜 1 {'安靜': 1}
美食 2 {'好吃': 1, '美食': 1}
有趣 2 {'熱鬧': 0.4160169804491255, '有意思': 0.5553046

# [9] 只印出景點 跟所屬的概念

In [7]:
##### [9] #####

#只印出所屬概念
for g in DictResult:
    print '【',g,'】',
    for u in DictResult[g]['概　念']:
        print u,
    print '\n'


【 新光摩天大樓 】 

【 松山慈祐宮 】 古老 購物 信仰 運動 壯觀 新奇 美食 有趣 

【 台北藝術大學 】 藝文 

【 捷運北門站 】 懷舊 

【 蔡瑞月舞蹈研究社 】 有趣 脫俗 

【 永康街青田街 】 漂亮 古老 樂活 脫俗 安靜 美食 有趣 文青 輕鬆 

【 梅庭 】 壯觀 脫俗 乾淨 

【 中研院歷史文物陳列館 】 特產 壯觀 

【 告白巷 】 髒亂 漂亮 壯觀 有趣 安靜 

【 西門町商圈 】 漂亮 古老 購物 遊玩 美食 有趣 藝文 

【 台北故事館 】 漂亮 文青 懷舊 

【 台北市孔廟 】 有趣 信仰 懷舊 

【 台北霞海城隍廟 】 特產 信仰 美食 

【 台灣高等法院 】 

【 四平公園 】 有趣 購物 安靜 美食 漂亮 

【 松山彩虹橋 】 漂亮 購物 藝文 

【 閻錫山墓 】 

【 台北琴道館 】 懷舊 

【 清水巖祖師廟 】 古老 信仰 髒亂 安靜 有趣 文青 

【 滾石國際音樂有限公司 】 安靜 藝文 

【 台北龍山寺 】 文青 壯觀 信仰 購物 有趣 

【 國父紀念館 】 特產 有趣 文青 壯觀 脫俗 

【 台北車站 】 美食 有趣 購物 壯觀 乾淨 

【 象山 】 漂亮 運動 安靜 髒亂 壯觀 有趣 藝文 

【 延平河濱公園 】 漂亮 安靜 

【 外交部 】 壯觀 安靜 文青 遊玩 

【 台北關渡宮 】 漂亮 古老 信仰 壯觀 美食 有趣 

【 台北101 】 漂亮 古老 購物 安靜 美食 特產 有趣 

【 富陽自然生態公園 】 樂活 信仰 乾淨 

【 世新大學 】 特產 安靜 美食 

【 北投文物館 】 安靜 文青 脫俗 

【 芝山文化生態綠園 】 購物 

【 撫台街洋樓 】 

【 饒河夜市足沐腳底按摩 】 文青 購物 輕鬆 

【 忠烈祠 】 有趣 信仰 脫俗 

【 寶藏巖國際藝術村 】 漂亮 有趣 藝文 

【 公館水岸公園 】 購物 

【 北投公園 】 古老 乾淨 樂活 安靜 有趣 文青 輕鬆 

【 鳳甲美術館 】 快樂 安靜 脫俗 壯觀 特產 有趣 

【 美麗華百樂園 】 漂亮 快樂 購物 有趣 脫俗 

【 慈諴宮 】 文青 信仰 購物 

【 台北市議會圖書館 】 有趣 脫俗 

【 天后宮 】 文青 壯觀 信仰 

【 自來水園區 】 有趣

# 延伸 A : 寫成JSon檔
DictResult : [字典]。

In [12]:
import json
json_str = json.dumps(DictResult)
#print json_str
with open('AttrTest_0119.json','w') as jfid:
    jfid.write(json_str)

# 延伸 B : 寫進MongoDB

In [14]:
import glob                                 #glob套件:讀取資料夾內所有檔案
import pymongo
import json
from pymongo import MongoClient
client = MongoClient('localhost', 27017)                            #設定client端，參數為MongoClient('連線端位置','連線端port位')
db = client['attr-2016-01-19']                                               #設定連接的資料庫
collection = db['Attr']                                    #設定連接的collection 
count = 0


with open('AttrTest_0119.json', 'r') as f:
    try: 
        patents = json.load(f)                                 #讀取每筆json檔內容
        postid = collection.insert_one(patents)                #將檔案插入mongoDB
        print "inserted with id" , postid                      
        count += 1
    except:
        print 'Wrong!'
print count  

inserted with id <pymongo.results.InsertOneResult object at 0x000000003B46EE10>
1
